In [2]:
from dotenv import load_dotenv
import os
import requests
import base64
from datetime import datetime
import json

def get_Activities():
    url = "https://intervals.icu/api/v1/athlete/0/activities?oldest=2020-12-01&newest=2027-12-31"
    
    load_dotenv('APIKEY2.env')
    username = os.getenv("USERNAME")
    password = os.getenv("PASSWORD")
    
    if not username or not password:
        print("Missing credentials.")
        return None

    credentials = base64.b64encode(f"{username}:{password}".encode()).decode()
    
    headers = {
        "Authorization": f"Basic {credentials}",
        "Accept": "application/json"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

activities = get_Activities()

if activities:
    if len(activities) > 0:
        last_activity = activities[0]
        try:
            activity_date = datetime.fromisoformat(last_activity['start_date_local'].replace('Z', '+00:00'))
            print(f"\nLast activity details:")
            print(f"Date: {activity_date}")
            print(f"Distance value: {last_activity.get('distance')}")
            print(f"ICU Distance value: {last_activity.get('icu_distance')}")
            print(f"Type: {last_activity.get('type', 'N/A')}")
            
            # Calculate today's activities
            today = datetime.now().date()
            total_today_km = 0.0
            rode_today = False
            
            # Check each activity for today and print raw values
            for activity in activities:
                activity_date = datetime.fromisoformat(activity['start_date_local'].replace('Z', '+00:00'))
                if activity_date.date() == today:
                    # Try icu_distance first, fall back to distance if not available
                    raw_distance = activity.get('icu_distance', activity.get('distance', 0))
                    print(f"Today's activity raw distance: {raw_distance}")
                    total_today_km += raw_distance
                    rode_today = True
                    
        except Exception as e:
            print(f"Error parsing activity: {e}")
            print("Raw activity data:", last_activity)

    # Save to data.json with raw distance value
    output = {
        "rode_today": rode_today,
        "distance_km": total_today_km
    }

    with open("data.json", "w") as f:
        json.dump(output, f)

    print("✔️ data.json updated.")
else:
    print("❌ Could not get activities.")


Last activity details:
Date: 2025-06-12 19:28:00
Distance value: None
ICU Distance value: None
Type: N/A
Today's activity raw distance: 0
✔️ data.json updated.
